## Setup

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import sklearn

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, train_test_split

## Email data

In [ ]:
from sklearn.linear_model import LogisticRegression

email = pd.read_csv(
  'https://sta663-sp22.github.io/slides/data/email.csv'
)[ [
  'spam', 'exclaim_mess', 'format', 'num_char', 'line_breaks', 'number'
] ]

email_dc = pd.get_dummies(email)
email_dc


y = email_dc.spam
X = email_dc.drop('spam', axis=1)

m = LogisticRegression(fit_intercept = False).fit(X, y)

### Demo 1 - DescisionTreeClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

### Demo 2 - SVC
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html|

### Exercise 1

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits(as_frame=True)
X, y = digits.data, digits.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, shuffle=True, random_state=1234
)